In [2]:
import numpy as np
import cv2 
from matplotlib import pyplot as plt
import os
import glob

In [3]:
def ssd(vecs1, vecs2, window_size):
    # print(vecs1.shape , vecs1)
    # print(vecs2.shape, vecs2)
    vecs1 = vecs1.flatten().reshape((2*window_size+1)**2, 3).astype(int)
    # print(vecs1)
    vecs2 = vecs2.flatten().reshape((2*window_size+1)**2, 3).astype(int)
    # print(vecs2)
    
    output = np.sum([np.sum((x)**2) for x in (vecs1-vecs2)])

    return output

Looping
---

Testing approach where we loop over every pixel. Way too slow. Takes about 1 min to do the calculation for a single row of x values in the given image

In [4]:

base_path  = os.getcwd() + "/Dataset/"
pathL = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg'
pathR = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg'

print(base_path + pathL)

window_size = 1 ## This is not the size of the actual window, our window looks like this [window_size :pixel of interest: window_size], so the size of the whole window is (2*window size)+1


left = cv2.imread(base_path + pathL)
right = cv2.imread(base_path + pathR)

print("Shape before padding ", left.shape)
print("Shape before padding ", right.shape)

left = cv2.copyMakeBorder(left, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)
right = cv2.copyMakeBorder(right, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)


# plt.imshow(left)
# plt.axis('off')
# plt.show()
# plt.imshow(right)
# plt.axis('off')
# plt.show()


print("Shape after padding ", left.shape)
print("Shape after padding ", right.shape)

x = left.shape[1] 
y = left.shape[0]


disparity = []

for i_y in range(1,2):
# for i_y in range(window_size, y-window_size):

    print(i_y-window_size)


    right_line_nbhds = []

    print("Before right x nbhds created")
    ## Create all the pixel neighbourhoods for the line in the right image
    for i_x2 in range(window_size, x-window_size):
            
        nbhd_right_img = np.array(right[i_y-window_size:i_y+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int) 
        right_line_nbhds.append(nbhd_right_img)

    print("After right x nbhds created")

    right_line_nbhds = np.array(right_line_nbhds)

    print("nbhds ",len(right_line_nbhds))

    ## For each x in the left image, compare it to every pixel neighbourhood in the right image (using SSD here) and take the smallest SSD
    for i_x in range(window_size, x-window_size):
        
        x_left = left[i_y-window_size:i_y + window_size + 1,i_x - window_size:i_x+window_size+1,:] ## Left pixel neighbourhood
        nbhd_ssd = np.array([ssd(x_left,r, window_size) for r in right_line_nbhds]) ## Generate all SSDs for left pixel (1 per right pixel nbhd) 
        match_r_img = np.argmin(nbhd_ssd) ## Take minimum ssd as the closest match       
        pixel_disparity = (i_x - window_size) - match_r_img ## Calculate disparity

        # print(i_x, match_r_img,pixel_disparity, nbhd_ssd )
        # print(i_x, match_r_img)

        disparity.append(pixel_disparity)

    
print(disparity)

        




/Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
Shape before padding  (400, 881, 3)
Shape before padding  (400, 881, 3)
Shape after padding  (402, 883, 3)
Shape after padding  (402, 883, 3)
0
Before right x nbhds created
After right x nbhds created
nbhds  881


KeyboardInterrupt: 

Vectorized Version 1
--- 

This brings the time down to about 2.5 seconds per row. Still not fast enough.

The way this method works is that for a given row and all the neighbourhoods related to that row, it turns them into a vector and performs vector operations to calculate our disparity. 

**The next step is to vectorize the whole image and perform these operations.**

The general idea is that each left pixel neighbourhood needs to be compared to every pixel neighbourhood on the same horizontal line in the right image. To do this for a single pixel neighbourhood in the left image we do the following. 

Let the left pixel neighbourhood be denoted as LX1,LY1 (which denotes the x,y coords of the pixel of focus)

We take (LX1,LY1) and tile it so that we have [(LX1,LY1) , (LX1,LY1) ...] with the neighbourhood repeated 881 times (horizontal length for the test image in this case). We then create the right neigbourhood which looks like [(RX1, RY1), (RX1, RY2) ... (RX1, RY881)]

now this is for a single instance, and we want to do the whole line at once so then we have 

[  [(LX1,LY1) , (LX1,LY1) ...] , [(LX1,LY2) , (LX1,LY2) ...] .... [(LX1,LY881) , (LX1,LY881) ...] ] as our left nbhd collection and, 

[(RX1, RY1), (RX1, RY2) ... (RX1, RY881)] repeated 881 times for our right neighbourhood collection. 

We use these to do a bunch of vector operations to calculate disparity. 


In [76]:

base_path  = os.getcwd() + "/Dataset/"
pathL = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg'
pathR = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg'

print(base_path + pathL)

window_size = 3 ## This is not the size of the actual window, our window looks like this [window_size :pixel of interest: window_size], so the size of the whole window is (2*window size)+1


left = cv2.imread(base_path + pathL)
right = cv2.imread(base_path + pathR)

print("Shape before padding ", left.shape)
print("Shape before padding ", right.shape)

left = cv2.copyMakeBorder(left, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)
right = cv2.copyMakeBorder(right, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)



print("Shape after padding ", left.shape)
print("Shape after padding ", right.shape)

x = left.shape[1] 
y = left.shape[0]


disparity = []

for i_y in range(1,10):
# for i_y in range(window_size, y-window_size):
    print(i_y-window_size)



    right_line_nbhds = []
    # print("Before right x nbhds created")

    ## Calculate all the pixel neighbourhoods for the row in the right image
    for i_x2 in range(window_size, x-window_size):
        nbhd_right_img = np.array(right[i_y-window_size:i_y+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int).flatten() ##.reshape(1,window_size**2)
        # print("right nbhd ", nbhd_right_img)
        right_line_nbhds.append(nbhd_right_img)


    right_line_nbhds = np.array(right_line_nbhds).flatten()

    right_line_nbhds = np.tile(right_line_nbhds, x-2*window_size) ## This replicates the right neighbourhoods


    left_line_nbhds = []

    # print("Before left x nbhds created")

    


    for i_x2 in range (window_size, x - window_size):

        nbhd_left_img = np.array(left[i_y-window_size:i_y+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int).flatten()
        nbhd_left_img = np.tile(nbhd_left_img, x - 2*window_size) ## Replicate that specific nbhd 881 times in this case
        left_line_nbhds.append(nbhd_left_img)


    left_line_nbhds = np.array(left_line_nbhds).flatten()

    # print(len(left_line_nbhds))

    flat = ((left_line_nbhds-right_line_nbhds)**2)

    step = ((2*window_size+1)**2)*3 ## Number of pixels in window * number of colour channels(3)
    waga = np.array([flat[i:i+step].sum() for i in range(0,len(flat), 27)])
    ssd = np.array([np.argmin(waga[i:i + (x - 2*window_size)]) for i in range (0, (x - 2*window_size)**2, (x - 2*window_size) )])

    row_disparity = ssd - np.linspace(0, x - (2*window_size + 1) ,num = (x-2*window_size)).astype(int)
    disparity.append(row_disparity)
 


        
disparity




/Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
Shape before padding  (400, 881, 3)
Shape before padding  (400, 881, 3)
Shape after padding  (402, 883, 3)
Shape after padding  (402, 883, 3)
0
1
2


KeyboardInterrupt: 

In [20]:
print(len(disparity[0]))

881


Vectorized Version 2
--- 

Not done yet

In [11]:




base_path  = os.getcwd() + "/Dataset/"
pathL = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg'
pathR = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg'

print(base_path + pathL)

window_size = 1 ## This is not the size of the actual window, our window looks like this [window_size :pixel of interest: window_size], so the size of the whole window is (2*window size)+1


left = cv2.imread(base_path + pathL)
right = cv2.imread(base_path + pathR)

print("Shape before padding ", left.shape)
print("Shape before padding ", right.shape)

left = cv2.copyMakeBorder(left, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)
right = cv2.copyMakeBorder(right, window_size, window_size, window_size, window_size, cv2.BORDER_CONSTANT, None, value = 0)



print("Shape after padding ", left.shape)
print("Shape after padding ", right.shape)

x = left.shape[1] 
y = left.shape[0]

# print(window_size, x-window_size)

disparity = []

# for i_y in range(1,10):
# for i_y in range(window_size, y-window_size):
    # print(i_y-window_size)



right_line_nbhds = []
print("Before right x nbhds created")

## Calculate all the pixel neighbourhoods for the entire image
# for i_y2 in range (window_size, y - window_size):
for i_y2 in range (window_size, window_size + 50):

    print(i_y2)

    for i_x2 in range(window_size, x-window_size):
        nbhd_right_img = np.array(right[i_y2-window_size:i_y2+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int).flatten() ##.reshape(1,window_size**2)
        # print("right nbhd ", nbhd_right_img)
        right_line_nbhds.append(nbhd_right_img)

    right_row_nbhds = np.array(right_line_nbhds).flatten()
    right_line_nbhds.append( np.tile(right_row_nbhds, x-2*window_size) ) 
    
    
print("After right x nbhds created")



right_line_nbhds = np.array(right_line_nbhds).flatten()

# right_line_nbhds = np.tile(right_line_nbhds, x-2*window_size) ## This replicates the right neighbourhoods


left_line_nbhds = []

# print("Before left x nbhds created")

for i_y2 in range (window_size, window_size + 50):

    for i_x2 in range (window_size, x - window_size):

        nbhd_left_img = np.array(left[i_y2-window_size:i_y2+window_size+1, i_x2-window_size:i_x2+window_size+1,:]).astype(int).flatten()
        
        # print("left nbhd ", nbhd_left_img)

        nbhd_left_img = np.tile(nbhd_left_img, x - 2*window_size)

        left_line_nbhds.append(nbhd_left_img)


left_line_nbhds = np.array(left_line_nbhds).flatten()

print(len(left_line_nbhds))
print(len(right_line_nbhds))

#     # print("left " ,left_line_nbhds[0:5])

#     # left_line_nbhds = np.array(left_line_nbhds).flatten()

#     # left_line_nbhds = np.repeat(left_line_nbhds, x)
#     # right_line_nbhds = np.tile(right_line_nbhds, (x,1))


#     # print("After left x nbhds created")

#     # print("right ", right_line_nbhds[0:5])
#     # print("left " ,left_line_nbhds)

#     # print("right ", right_line_nbhds.shape)
#     # print("left " ,left_line_nbhds.shape)

#     # print("sub ",(left_line_nbhds-right_line_nbhds)[0:54])

#     # print("sub2 ",((left_line_nbhds-right_line_nbhds)**2)[0:54])

#     # # print("sub2 ",((left_line_nbhds-right_line_nbhds)**2).flatten()[0:27])

flat = ((left_line_nbhds-right_line_nbhds)**2)

#     # print(flat.shape)

#     # print(flat[0:27])
#     # print(flat[27:54])
#     # print(flat[54:81])

#     # print("flat ", flat[0:27].sum())
#     # print("flat ", flat[27:54].sum())
#     # print("flat ", flat[54:81].sum())

#     # print(len(flat)/27)

# step = ((2*window_size+1)**2)*3 ## Number of pixels in window * number of colour channels(3)
# waga = np.array([flat[i:i+step].sum() for i in range(0,len(flat), 27)])

# #     # print(waga.shape)
# #     # np.argmin(flat) 

# ssd = np.array([np.argmin(waga[i:i + (x - 2*window_size)]) for i in range (0, (x - 2*window_size)**2, (x - 2*window_size) )])

#     # print(ssd.shape)

#     # print(ssd)
#     row_disparity = ssd - np.linspace(0, x - (2*window_size + 1) ,num = (x-2*window_size)).astype(int)
#     disparity.append(row_disparity)
 


    

#     # print(waga)

#     # for l_x in left_line_nbhds:


    


#     # print("nbhds ",len(right_line_nbhds))

#     # # for i_x in range(10):
#     # for i_x in range(window_size, x-window_size):
        
        

#     #     x_left = left[i_y-window_size:i_y + window_size + 1,i_x - window_size:i_x+window_size+1,:]

#     #     # nbhd_right_img = right[i_y-window_size:i_y+window_size+1, i_x-window_size:i_x+window_size+1,:] ## nbhd = neighbourhood

#     #     nbhd_ssd = np.array([ssd(x_left,r, window_size) for r in right_line_nbhds]) ## Generate ssd for neighbourhood



#     #     match_r_img = np.argmin(nbhd_ssd) ## Take minimum ssd as the closest match 
#     #     pixel_disparity = (i_x - window_size) - match_r_img ## Calculate disparity
        
#     #     print(i_x)

#     #     disparity.append(pixel_disparity)


        
# disparity


/Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
Shape before padding  (400, 881, 3)
Shape before padding  (400, 881, 3)
Shape after padding  (402, 883, 3)
Shape after padding  (402, 883, 3)
Before right x nbhds created
1
2
3
4
5
6
7
8


/var/folders/b4/7ptvpcjn0jq8_vq3_fjnmxrc0000gn/T/ipykernel_7957/862832702.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  right_row_nbhds = np.array(right_line_nbhds).flatten()


9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
After right x nbhds created


/var/folders/b4/7ptvpcjn0jq8_vq3_fjnmxrc0000gn/T/ipykernel_7957/862832702.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  right_line_nbhds = np.array(right_line_nbhds).flatten()


1047817350
44100


ValueError: operands could not be broadcast together with shapes (1047817350,) (44100,) 

Function that implemments vectorized version 1

In [27]:

## ws stand for window size, window size component 
def img_disparity(img_left, img_right, ws_component):

    img_left = cv2.copyMakeBorder(left, ws_component, ws_component, ws_component, ws_component, cv2.BORDER_CONSTANT, None, value = 0)
    img_right = cv2.copyMakeBorder(right, ws_component, ws_component, ws_component, ws_component, cv2.BORDER_CONSTANT, None, value = 0)

    x = img_left.shape[1] 
    y = img_left.shape[0]

    print(y)
    print(x)

    disparity = []

    # for i_y in range(1,10):
    for i_y in range(ws_component, y-ws_component):

        # print(i_y)
        
        right_line_nbhds = []

        ## Calculate all the pixel neighbourhoods for the row in the right image
        for i_x2 in range(ws_component, x-ws_component):
            nbhd_right_img = np.array(img_right[i_y-ws_component:i_y+ws_component+1, i_x2-ws_component:i_x2+ws_component+1,:]).astype(int).flatten() ##.reshape(1,window_size**2)
            right_line_nbhds.append(nbhd_right_img)

        


        right_line_nbhds = np.array(right_line_nbhds).flatten()
        right_line_nbhds = np.tile(right_line_nbhds, x-2*ws_component) ## This replicates the right neighbourhoods
        
        # print(right_line_nbhds[0:100])

        # print("rhnbdhs " , len(right_line_nbhds))

        left_line_nbhds = []

        for i_x2 in range (ws_component, x - ws_component):

            # print(i_x2)

            
            # print(img_left[i_x2, i_y,:])
            # print(img_left[i_x2-1, i_y,:])
            # print(img_left[i_x2, i_y-1,:])

            nbhd_left_img = np.array(img_left[i_y-ws_component:i_y+ws_component+1, i_x2-ws_component:i_x2+ws_component+1,:]).astype(int).flatten()
            nbhd_left_img = np.tile(nbhd_left_img, x - 2*ws_component) ## Replicate that specific nbhd by the amount of pixels we consider in a row
            left_line_nbhds.append(nbhd_left_img)




        left_line_nbhds = np.array(left_line_nbhds).flatten()

        # print(left_line_nbhds[0:100])

        # print("left nbdhs " , len(left_line_nbhds))

        difference = ((left_line_nbhds-right_line_nbhds)**2)

        # print(difference[0:100])

        step = ((2*ws_component+1)**2)*3 ## Number of pixels in window * number of colour channels(3)

        ssds = np.array([difference[i:i+step].sum() for i in range(0,len(difference), step )] ) ## Sums the squared difference for intervals of length step 

        min_ssd = np.array([np.argmin(ssds[i:i + (x - 2*ws_component)]) for i in range (0, (x - 2*ws_component)**2, (x - 2*ws_component) )])

        row_disparity = min_ssd - np.linspace(0, x - (2*ws_component + 1) ,num = (x-2*ws_component)).astype(int)

        disparity.append(row_disparity)
    
            
    return disparity


In [26]:
base_path  = os.getcwd() + "/Dataset/"
pathL = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg'
pathR = '2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg'

print(base_path + pathL)

left = cv2.imread(base_path + pathL)
right = cv2.imread(base_path + pathR)

print(left.shape)

disparity = img_disparity(left, right, ws_component = 1) ## Note that total window size is (2*ws_component + 1)

/Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
(400, 881, 3)
402
883
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
2

KeyboardInterrupt: 

In [69]:
strh = "2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg_ws3"
np.save(strh,disparity)

In [56]:
np.save("2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg_ws3",disparity) 

In [36]:
os.getcwd()

'/Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1'

In [39]:
x = np.load("2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg_ws3.npy")

In [6]:
import time

In [54]:
disparity

[array([   0,   65,   65,   65,   -3,   -4,   -5,   -6,   -7,   -8,   -9,
          27,   26,   25,   24,   23,   22,   21,   20,  -17,  -17,  -16,
          -8,  -16,  -11,  -13,    9,  -23,    9,  -18,    6,    6,    6,
           5,  104,  103,  102,  101,    2,   29,   28,    0,    0,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -2,   -1,   -2,   -2,   -2,
          -2,   -2,   -2,   -2,   -1,   -2,   -2,   -2,   -2,   -2,   -2,
          -2,   -2,   -3,   -2,   -3,   -3,  -71,  -72,  -73,  -74,  -75,
          -7,   60,   47,   46,   44,   43,    0,   -1,   24,  -51,  -52,
         -53,  -53,  -54,  -55,  -56,  -57,  -57,  -59,  -60,  -61,  -61,
         -87,  -88,  -88,  -89,  -90,  -91,  -92,  -92,  -93,  -94,  -73,
         -74,  -73,  -74,  -75,  -76,   11,   10,    9,    8,  -85,  -10,
         510,  510,  -10,  -10,  -10,  -10,  224,   -2,   -3,    6,    6,
          -7,   -7,  -48, -117, -100,  -98,  -99,  -99,   -2,   -1,   -2,
          -2,   -1,   -1,   -1,   -1, 

In [28]:
data_dir = os.getcwd() + '/Dataset/*.jpg'
img_files = glob.glob(data_dir)

img_files.sort() ## Sorts images so <img>left and <img>right are next to eachother. 

window_size_components = [1, 3, 6] ## Remember window size component is 2*window_size_component+1

all_disparties_ws3 = []

i = 0 
while i < len(img_files):


    left = cv2.imread(img_files[i])
    right = cv2.imread(img_files[i+1])

    # print(left.shape)
    # print(right.shape, "\n")

    print("left image ", img_files[i])
    print("right image " ,img_files[i+1], "\n")


    # for w in window_size_components:

    start = time.time()

    disparity = img_disparity(left, right, ws_component = 1)

    end = time.time()

    print("Time taken for ws = {} ".format(3) , end - start)

    # disc = np.array([1,2,3])
    # name = img_files[i] + "_ws" + str(w*2+1) + ".npy"

    # print(name)

    all_disparties_ws3.append(disparity)

    print("Completed x iterations", len(all_disparties_ws3))

    # print(filename)
    # np.save(file = name,arr = disc) 



    

    i += 2

    


left image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
right image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg 

402
883
Time taken for ws = 3  821.6037938594818
Completed x iterations 1
left image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-13-38-366-left.jpg
right image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-13-38-366-right.jpg 

402
883
Time taken for ws = 3  812.9330103397369
Completed x iterations 2
left image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-14-54-205-left.jpg
right image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-0

In [32]:
len(all_disparties_ws3[0])

400

In [30]:
np.save("ws3_results", all_disparties_ws3)

In [29]:

data_dir = os.getcwd() + '/Dataset/*.jpg'
img_files = glob.glob(data_dir)

img_files.sort() ## Sorts images so <img>left and <img>right are next to eachother. 

window_size_components = [1, 3, 6] ## Remember window size component is 2*window_size_component+1

all_disparties_ws5 = []

i = 0 
while i < len(img_files):


    left = cv2.imread(img_files[i])
    right = cv2.imread(img_files[i+1])

    # print(left.shape)
    # print(right.shape, "\n")

    print("left image ", img_files[i])
    print("right image " ,img_files[i+1], "\n")


    # for w in window_size_components:

    start = time.time()

    disparity = img_disparity(left, right, ws_component = 2)

    end = time.time()

    print("Time taken for ws = {} ".format(3) , end - start)

    # disc = np.array([1,2,3])
    # name = img_files[i] + "_ws" + str(w*2+1) + ".npy"

    # print(name)

    all_disparties_ws5.append(disparity)

    print("Completed x iterations", len(all_disparties_ws5))

    # print(filename)
    # np.save(file = name,arr = disc) 



    

    i += 2


left image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg
right image  /Users/oliverbestel/Desktop/Documents/Work/COMP90086/Tuts/P1/CV_Project/Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg 

404
885


KeyboardInterrupt: 